Programmieren 3 - Grafische Benutzerschnittstellen

Peter Rösch, Fakultät für Informatik

Hochschule Augsburg, 2023/2024

In [ ]:
# Speichern des urspruenglichen Verzeichnisses
%bookmark myNotebookDirectory

# Nachträge und Ergänzungen

## Pylint und nicht genutzte Zählvariable

Wird das Ergebnis der Iteration in einer Schleife nicht genutzt, so gibt es Punktabzug von *pylint*.

**Lösung:** Nennen Sie die Variable "_"

    # Version mit pylint-Meldung
    for i in range(10):
        print('Ausgabe ohne i')

    # Version ohne Punktabzug
    for _ in range(10):
        print('Ausgabe ohne i')

## numpy: Prototypen-Entwicklung - Schiefer Wurf

**Kontext:** (Fast) alle Informatik-Projekte sind interdisziplinär, so dass Sie sich künftig recht häufig in Themen außerhalb der Informatik einarbeiten werden. Entscheidend ist, dass Sie die Aufgabe systematisch analysieren und offen Fragen mit Kunden bzw. Experten klären.

**Aufgabe:** Berechnen Sie die Bahnkurven für den schiefen Wurf im Vakuum auf der Erde für verschiedene Startwinkel und Startgeschwindigkeiten. Stellen Sie das Ergebnis grafisch dar.

**Problem verstehen:** Gibt es Fragen?

**Recherche / Theorie**: </br>
Es wirkt die Gewichtskraft in negative z-Richtung
$$F_z = -m g, \; g = 9.81 \frac{m}{s^2}$$
Daraus ergibt sich die konstante Beschleunigung in $z$ -Richtung
$$a_z = -g$$
also
$$\vec{a} = \left( \begin{array}{c} 0 \\ 0 \\ -g \end{array}\right)$$

Die Bewegung findet in einer Ebene statt. Entscheidung: Wir betrachten die $xz$-Ebene.
Vorgegeben ist eine Anfangsgeschwindigkeit $|v_0|$, ein Startwinkel $\alpha$ und ein Startpunkt $\vec{x}_0$
Daraus folgt:
$$\vec{v_0} = \left( \begin{array}{c} v_0 \cos(\alpha) \\ 0 \\ v_0 \sin(\alpha) \end{array}\right)$$
Iteration:
$$\vec{x}_{n+1} = \vec{x}_n + \vec{v} \, \Delta t + \vec{a} \, \frac{\Delta t^2}{2}$$
$$\vec{v}_{n+1} = \vec{v}_n +  \vec{a} \, \Delta t$$

**Teilaufgaben:**
    
* Initialisierung des Systems (Anfangsbedingungen)
* Berechnung und Speicherung der Bahnkurve
* Visualiseriung der Ergebnisse

**Ansätze und Entscheidungen:** (Algorithmen und Datenstrukturen)

* Verwendung von *numpy*-Arrays für die Vektoren $\vec{x}, \vec{v}, \vec{a}$
* Schrittweise Simulation unter Verwendung der oben gegebenen Formeln
* Speichern der Position als Funktion der Zeit in einem *numpy*-array
* Anzeige mit *matplotlib*


In [ ]:
# Initialisierung
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib ipympl
#
# Start
x0 = np.array([20, 0, 10], np.float32)
v0_abs = 20
alpha = 45
#alpha = 0
alpha_rad = math.radians(alpha)
g = 9.81
a = np.array([0, 0, -g], np.float32)
delta_t = 0.01
x = x0.copy()
v = np.array([v0_abs*math.cos(alpha_rad), 0, 
              v0_abs*math.sin(alpha_rad)],
              np.float32)

In [ ]:
#
# Liste fuer die Zwischenpositionen
pos_list = []
#
# Berechnung
while x[2] >= 0.0:
    x = x + v * delta_t + a * delta_t**2 / 2
    v = v + a * delta_t
    pos_list.append(x)

# Umwandlung in ein zweidimensionales Numpy-Array
pos_array = np.array(pos_list, dtype=np.float32)
#
# Anzeige, z als Funktion von x
ax = plt.figure().add_subplot(111)
ax.plot(pos_array[:, 0], pos_array[:, 2])

## Interaktion im Notebook

### Erstellen einer geeigneten Funktion für die Berechnung

In [ ]:
import math

G_EARTH = 9.81


def projectile_simulation_vacuum(
    initial_height_m: float,
    initial_absolute_velocity_m_s: float = 20,
    angle_deg: float = 20,
    time_step_s: float = 0.01,
) -> np.ndarray:
    position = np.array([0, 0, initial_height_m], np.float32)
    # math requires angles in radians
    alpha_rad = math.radians(angle_deg)
    velocity = np.array(
        [
            initial_absolute_velocity_m_s * math.cos(alpha_rad),
            0,
            initial_absolute_velocity_m_s * math.sin(alpha_rad),
        ],
        np.float32,
    )
    acceleration = np.array([0, 0, -G_EARTH], dtype=np.float32)
    position_list = []
    while position[2] >= 0.0:
        position = (
            position
            + velocity * time_step_s
            + 0.5 * acceleration * time_step_s**2
        )
        velocity = velocity + acceleration * time_step_s
        position_list.append(position)
    return np.array(position_list, dtype=np.float32)

### Vertrauensbildende Massnahme

In [ ]:
trajectory = projectile_simulation_vacuum(
    initial_height_m=10,
    initial_absolute_velocity_m_s=20,
    angle_deg=45,
    time_step_s=0.01,
)
ax = plt.figure().add_subplot(111)
ax.plot(trajectory[:, 0], trajectory[:, 2])

Jetzt wollen wir erreichen, dass eine Grafik ausgegeben wird und die Kurve sich während der Interaktion ändert.
Quelle: [stackoverflow](https://stackoverflow.com/questions/44329068/jupyter-notebook-interactive-plot-with-widgets)

In [ ]:

trajectory = projectile_simulation_vacuum(
    initial_height_m=10,
    initial_absolute_velocity_m_s=20,
    angle_deg=45,
    time_step_s=0.01,
)

In [ ]:
ax = plt.figure().add_subplot(111)

from ipywidgets import interact


def update(
    initial_height_m=10,
    initial_absolute_velocity_m_s=20,
    angle_deg=45,
    time_step_s=-0.01,
):
    trajectory = projectile_simulation_vacuum(
        initial_height_m,
        initial_absolute_velocity_m_s,
        angle_deg,
        time_step_s,
    )
    ax.clear()
    ax.set_xlabel("x / m")
    ax.set_ylabel("z / m")
    title = (
        f"h0={initial_height_m:.1f}, "
        f"v0={initial_absolute_velocity_m_s:.1f}, alpha={angle_deg:.1f}, "
        f"x_max={trajectory[-1, 0]:.1f}"
    )
    ax.set_title(title)
    ax.plot(trajectory[:, 0], trajectory[:, 2])

In [ ]:
interact(
    update,
    initial_height_m=(0, 20, 1),
    initial_absolute_velocity=(5, 30, 1),
    angle_deg=(0, 90, 1),
    time_step_s=(0.01, 1.01, 0.05),
)

## Gruppenarbeit (15 Minuten)

Erweitern Sie die oben gegebene Simulation des schiefen Wurfs, so dass der Luftwiderstand berücksichtigt wird. Es soll eine Kugel der Masse 10 kg und variablem Radius $r$ (Einheit: m) bei $20^{\circ}$ auf Meereshöhe betrachtet werden. Die Parameter $\alpha$, $|v_0|$ und $r$ sollen über Schieberegler im Notebook (Stichwort *interact* ) modifizierbar sein. Wie ändert sich die Kurve mit wachsendem $r$?

**Hilfestellung:**
$$ \vec{F}_{\rm L} = - \frac{1}{2} \vec{v} \cdot |\vec{v}| \cdot A \cdot c_W \cdot \rho_{\rm Luft},\; A = \pi r^2 $$

Dabei bezeichnet $\rho_{\rm Luft}$ die Luftdichte und $c_W$ den Strömungswiderstandskoeffizienten. Wir setzen für die Kugel [$c_W = 0.5$](https://www.code-knacker.de/cw-wert.htm).

**Visualisierung:** [phet.colorado.edu](https://phet.colorado.edu/sims/html/projectile-motion/latest/projectile-motion_en.html)

# Grafische Benutzerschnittstellen - Ziele

* Studierende kennen die Grundprinzipien grafischer Benutzerschnittstellen.
* Teilnehmer(innen) können anspruchsvolle GUI-Anwendungen mit dem Qt Designer, PyQt5 und Python interaktiv erstellen und vom IPython-Notebook aus nutzen.
* Das vorgegebene Beispiel *galaxy_renderer* kann von Studierenden für das Semester-Projekt verwendet und ggf. modifiziert werden.

# Einführung und Wiederholung

Die Geschichte der grafischen Benutzerschnittstellen begann in den sechziger Jahren des letzten Jahrhunderts. Zusammenfassungen finden sich bei [Wikipedia](https://www.sitepoint.com/real-history-gui/) sowie [hier](http://www.sitepoint.com/real-history-gui). Bemerkenswert ist, dass der erste [Prototyp einer Computer-Maus](https://de.wikipedia.org/wiki/Maus_%28Computer%29) schon 1963 entstand.

Zentral für die GUI-Programmierung ist die Behandlung von Ereignissen ([Events](http://en.wikipedia.org/wiki/Event_%28computing%29)), die vom Benutzer durch Eingaben mit Maus, Tastatur etc. ausgelöst werden. Verschiedene GUI-Toolkits setzen unterschiedliche Mechanismen ein, um diese Ereignisse zu verarbeiten. 

Ausgehend vom bereits bekannten *ActionListener*-Interfaces (Java) wird der *Signal/Slot*-Mechanismus [PyQt](http://www.riverbankcomputing.com/software/pyqt/intro) eingeführt. Der letzte Abschnitt nutzt dann [Callback-Funktionen](http://en.wikipedia.org/wiki/Callback_%28computer_programming%29) im Zusammenhang mit [tkinter](https://wiki.python.org/moin/TkInter).

# Qt mit Python

## Pylint und die Bibliothek *PyQt5*

*pylint* hat Probleme, Namen in C-Erweiterungen wie *Qt* zu finden und erzeugt Fehlermeldungen. Eine mögliche Lösung ist, das Modul zu importieren und dann nach Namen zu suchen, siehe [Dokumentation](https://pylint.pycqa.org/en/latest/technical_reference/c_extensions.html).

**Konkret:** Erstellen Sie eine Datei ~/.pylintrc wie folgt:

In [ ]:
%%file ~/.pylintrc
[MASTER]
    extension-pkg-allow-list=PyQt5

## Qt - Grundlagen

Die Verarbeitung von Ereignissen in Qt basiert auf dem [Signal-Slot-Konzept](http://de.wikipedia.org/wiki/Signal-Slot-Konzept), wobei die Kern-Funktionalität in C++ implementiert ist. Das Paket [PyQt](http://www.riverbankcomputing.com/software/pyqt/intro) erlaubt die Nutzung der Qt-Software von Python aus. Eine sehr gute Einführung bietet das Buch [Rapid GUI Programming with Python and Qt](http://www.qtrac.eu/pyqtbook.html) von Mark Summerfield, das auch über O'Reilly verfügbar ist.

Qt lässt sich von unterschiedlichen [Programmiersprachen](http://en.wikipedia.org/wiki/List_of_language_bindings_for_Qt_5) aus nutzen und funktioniert auf vielen Plattformen.

## PyQt5

Die Integration von PyQt in IPython-Notebooks geschieht über die "magische Funktion" *%gui qt5*, wie in der [Dokumentation](http://ipython.readthedocs.io/en/stable/config/eventloops.html) nachzulesen ist. 

In [ ]:
%gui
%gui qt5
# zurueck ins Notebook-Verzeichnis
#  ansonsten wird die Datei exit.png nicht gefunden
%cd myNotebookDirectory

Das folgende Beispiel enthält viele wiederverwendbare "Bausteine":

In [ ]:
from PyQt5 import QtWidgets, QtCore
import sys


class QuitButton(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super(QuitButton, self).__init__(parent)
        self.setGeometry(300, 300, 250, 150)
        self.setWindowTitle("Icon")
        self.quit = QtWidgets.QPushButton("Quit", self)
        self.quit.setGeometry(10, 10, 60, 35)
        self.quit.clicked.connect(self.close)


qb = QuitButton()
qb.show()

**Bitte beachten:** Eine Vorlage für Anwendungen ausserhalb des Notebooks sieht so aus:

In [ ]:
%%file qt_demo.py
from PyQt5 import QtWidgets, QtCore
import sys

class QuitButton(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super(QuitButton, self).__init__(parent)
        self.setGeometry(300, 300, 250, 150)
        self.setWindowTitle('Icon')
        self.quit = QtWidgets.QPushButton('Quit', self)
        self.quit.setGeometry(10, 10, 60, 35)
        self.quit.clicked.connect(self.close)

if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    qb = QuitButton()
    qb.show()
    sys.exit(app.exec_())

Die Anwendung können Sie vom Terminal aus starten mit *python qt_demo.py*.

Außer der *QWidget*-Klasse gibt es weitere vordefinierte Elemente, die als Basis für eigene Anwendungen verwendet werden können. Beispiel: *QMainWindow*

In [ ]:
# source: http://zetcode.com/gui/pyqt5/menustoolbars/
from PyQt5 import QtWidgets, QtGui


class MainWindow(QtWidgets.QMainWindow):
    """
    MainWindows with symbol, tooltip etc.
    """

    def __init__(self):
        QtWidgets.QMainWindow.__init__(self)

        self.resize(350, 250)
        self.setWindowTitle("MainWindow")
        textEdit = QtWidgets.QTextEdit()
        self.setCentralWidget(textEdit)
        exit = QtWidgets.QAction(QtGui.QIcon("exit.png"), "&Exit", self)
        exit.setShortcut("Ctrl+Q")
        exit.setStatusTip("Exit application")
        exit.triggered.connect(self.close)
        self.statusBar()
        menubar = self.menuBar()
        file = menubar.addMenu("&File")
        file.addAction(exit)
        toolbar = self.addToolBar("Exit")
        toolbar.addAction(exit)


main = MainWindow()
main.show()

**Aufgaben (empfohlen):** 

1. Experimentieren Sie mit den Interaktions-Möglichkeiten, die Ihnen die Klasse MainWindow bietet und schauen Sie die Befehle, die unklar sind, in der [Dokumentation](http://pyqt.sourceforge.net/Docs/PyQt5) nach.
1. Ergänzen Sie das Beispiel um einen Menü-Punkt Ihrer Wahl.

Widgets können durch *Signals* und *Slots* miteinander verbunden werden:

In [ ]:
import sys
from PyQt5 import QtGui, QtWidgets, QtCore


class SigSlot(QtWidgets.QDialog):
    def __init__(self, parent=None):
        QtWidgets.QDialog.__init__(self)
        self.setWindowTitle("signal & slot")
        self.dial = QtWidgets.QDial()
        self.dial.setNotchesVisible(True)
        self.spinbox = QtWidgets.QSpinBox()
        self.layout = QtWidgets.QHBoxLayout()
        self.layout.addWidget(self.dial)
        self.layout.addWidget(self.spinbox)
        self.setLayout(self.layout)
        self.dial.valueChanged.connect(self.spinbox.setValue)
        self.spinbox.valueChanged.connect(self.dial.setValue)


qb = SigSlot()
qb.show()

Mit Signals und Slots können auch eigene Funktionen und Methoden mit Bedienelementen verknüpft werden.

In [ ]:
from PyQt5 import QtWidgets


class MoinButton(QtWidgets.QWidget):
    def __init__(self, parent=None):
        QtWidgets.QWidget.__init__(self)
        self.setGeometry(300, 300, 250, 150)
        self.setWindowTitle("Moin")
        self.moinButton = QtWidgets.QPushButton("Moin", self)
        self.moinButton.setGeometry(100, 10, 60, 35)
        self.moinButton.clicked.connect(self.moin)

    def moin(self):
        QtWidgets.QMessageBox.about(self, "", "MoinMoin")


qb = MoinButton()
qb.show()

**Aufgaben (empfohlen):**

1. Verschaffen Sie sich einen Überblick über [dieses Tutorial](http://zetcode.com/gui/pyqt5).
1. Ergänzen Sie die Klasse *SigSlot* um einen Schieberegler, der mit den anderen Elementen gekoppelt ist. Sobald Sie die Einstellung eines Elements modifizieren, sollen sich die anderen beiden Elemente anpassen.
1. Visualisieren Sie die Beziehungen zwischen den Widgets in der von Ihnen ergänzten Anwendung (Papier / Bleistift).

# Der Qt-Designer

## Erstellung einer GUI

Das Programm *designer* erlaubt es Ihnen, grafische Benutzerschnittstellen interaktiv zu erstellen und zu modifizieren.

**Aufgaben:**

1. Verschaffen Sie sich einen Überblick über die [Dokumentation des Qt-Designers](https://doc.qt.io/qt-5/qtdesigner-manual.html).
1. Welche wichtigen Gründe gibt es dafür, Benutzeroberflächen mit dem Designer zu entwerfen anstatt sie im Programm-Code zu definieren?

## Einbindung in eine Python-Anwendung

Folgende Applikation lädt eine mit dem Designer erstellte ui-Datei und nutzt die Signal/Slot-Verbindungen, die im Designer definiert wurden.

In [ ]:
%cd myNotebookDirectory

In [ ]:
from PyQt5 import QtWidgets, QtGui, uic


class UiDemo(QtWidgets.QDialog):
    # constructor
    def __init__(self):
        QtWidgets.QDialog.__init__(self)
        # load and show the user interface created with the designer.
        self.ui = uic.loadUi("qtDemo.ui")
        self.ui.show()


uiDemo = UiDemo()

In [ ]:
uiDemo.ui.myHorizontalSlider.setValue(20)

Das nächste Beispiel verbindet eine selbst definierte Methode mit einem Button-Signal:

In [ ]:
from PyQt5 import QtWidgets, QtGui, uic


class UiDemo(QtWidgets.QDialog):
    # constructor
    def __init__(self):
        QtWidgets.QDialog.__init__(self)

        # load and show the user interface from Designer.
        self.ui = uic.loadUi("qtDemo.ui")
        self.ui.show()

        # Connect up the button.
        self.ui.myPushButton.clicked.connect(self.printLcdNumber)

    # own function to print a number
    def printLcdNumber(self):
        number = self.ui.myHorizontalSlider.value()
        QtWidgets.QMessageBox.about(self, "", f"number: {number}")


uiDemo = UiDemo()

Mit der Bibliothek *matplotlib* können Funktionsgraphen in PyQt-Anwendungen eingebunden werden: 

In [ ]:
from PyQt5 import QtWidgets, uic
from numpy import arange, sin, cos, pi
from matplotlib.backends.backend_qt5agg import (
    FigureCanvasQTAgg as FigureCanvas,
)
from matplotlib.figure import Figure


class MatplotlibDemo(QtWidgets.QMainWindow):
    def __init__(self):
        QtWidgets.QMainWindow.__init__(self)
        self.ui = uic.loadUi("matplotlib_demo.ui")
        self.fig = Figure(figsize=(5, 4), dpi=100)
        self.figureCanvas = FigureCanvas(self.fig)
        self.figureCanvas.setParent(self.ui.drawWidget)

        self.axes = self.fig.add_subplot(111)
        self.figureCanvas.setSizePolicy(
            QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Expanding
        )
        self.figureCanvas.updateGeometry()
        self.ui.show()
        self.__zeichneFunktion(lambda x: 0 * x)
        self.ui.sinusKnopf.clicked.connect(self.zeichneSinus)
        self.ui.cosinusKnopf.clicked.connect(self.zeichneCosinus)
        self.ui.sincKnopf.clicked.connect(self.zeichneSinc)

    def __zeichneFunktion(self, f, yGrenzen=(-1.2, 1.2)):
        xGrenzen = (-5 * pi, 5 * pi)
        x = arange(xGrenzen[0], xGrenzen[1], 0.01)
        self.axes.clear()
        self.axes.plot(x, f(x))
        self.axes.set_xlim(xGrenzen)
        self.axes.set_ylim(yGrenzen)
        self.figureCanvas.draw()

    def zeichneSinus(self):
        self.ui.statusZeile.showMessage("Sinus-Kurve")
        self.__zeichneFunktion(sin)

    def zeichneCosinus(self):
        self.ui.statusZeile.showMessage("Cosinus-Kurve")
        self.__zeichneFunktion(cos)

    def zeichneSinc(self):
        self.ui.statusZeile.showMessage("Sinc-Kurve, sin(x)/x")
        self.__zeichneFunktion(lambda x: sin(x) / x, yGrenzen=(-0.3, 1.1))


matplotlibDemo = MatplotlibDemo()

**Aufgaben (empfohlen):**

1. Greifen Sie von einer Zelle des IPython-Notebooks auf Methoden des Objekts *uiDemo* zu, während die Anwendung läuft.
1. Ergänzen Sie die Benutzeroberfläche *qtDemo* im Designer um einen Button, der die Anwendung beendet.
1. Experimentieren Sie mit der Klasse *MatplotlibDemo*.

# Tkinter mit Python

## Tkinter - Grundlagen

Tkinter ist Teil der Python-Standard-Distribution und nutzt callback-Funktionen, um Widgets mit Ereignissen zu verbinden. Ein Tutorial finden Sie [hier](http://zetcode.com/gui/tkinter/). 

In [ ]:
%%file /tmp/calc.py
import tkinter
from tkinter import W, E, N, S
from tkinter import ttk

def calculate(*args):
    try:
        value = float(feet.get())
        meters.set((0.3048 * value * 10000.0 + 0.5)/10000.0)
    except ValueError:
        print('Value Error')
    
root = tkinter.Tk()
root.title("Feet to Meters")

mainframe = ttk.Frame(root, padding="3 3 12 12")
mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
mainframe.columnconfigure(0, weight=1)
mainframe.rowconfigure(0, weight=1)

feet = tkinter.StringVar()
meters = tkinter.StringVar()

feet_entry = ttk.Entry(mainframe, width=7, textvariable=feet)
feet_entry.grid(column=2, row=1, sticky=(W, E))

ttk.Label(mainframe, textvariable=meters).grid(
                                    column=2, row=2, sticky=(W, E))
ttk.Button(mainframe, text="Calculate", command=calculate).grid(
                                    column=3, row=3, sticky=W)

ttk.Label(mainframe, text="feet").grid(column=3, row=1, sticky=W)
ttk.Label(mainframe, text="is equivalent to").grid(column=1,
                                                   row=2, sticky=E)
ttk.Label(mainframe, text="meters").grid(column=3, row=2, sticky=W)

for child in mainframe.winfo_children(): child.grid_configure(
                                                    padx=5, pady=5)

feet_entry.focus()
root.bind('<Return>', calculate)

root.mainloop()

In [ ]:
%run /tmp/calc

**Aufgaben (freiwillig):**

1. Schauen Sie sich das [Tkinter-Tutorial](http://www.tkdocs.com/tutorial/index.html) an und verschaffen Sie sich einen Überblick über die Widgets, die Tkinter zur Verfügung stellt.
1. Vergleichen Sie die Vor- und Nachteile von tkinter und PyQt5 (Tabelle).

# Übungsaufgaben, Abgabe  31.10.2023 bzw. 02.11.2023

1. Vervollständigen Sie Ihre Anwendung *funktionsplotter* (Python-Paket, Kommandozeilen-Schnittstelle, html-Dokumentation).
1. Erstellen Sie mit dem Designer und PyQt5 eine GUI für den Funktionsplotter. Die Größe des Text-Ausgabefensters soll konstant 40 Zeichen breit und 31 Zeichen hoch sein. Der Funktionsterm sowie das Intervall, in der die Funktion ausgeben wird, soll durch geeignete Bedienelemente eingestellt werden können.
1. Semester-Projekt: Definieren Sie schriftlich, wie sich Ihr Biest in speziellen Situationen verhalten soll und demonstrieren Sie in einem Jupyter-Notebook, wie Sie das gewünschte Verhalten in Python umsetzen können. Finden Sie Situationen, die für Ihre Steuerung eine Herausforderung darstellen und analysieren Sie das Verhalten Ihrer Lösung in diesen Fällen.

# Überprüfung

1. Erklären Sie den Unterschied zwischen "Callback-Funktionen" und dem "Signal / Slot"-Mechanismus. (max. vier Sätze)
1. Welche Vorteile bietet die Kombination von IPython-Notebooks und PyQt5 für die interaktive Software-Entwicklung? (max. zwei Sätze)